In [1]:
import pandas as pd
import csv
import numpy as np
path = r"C:\\Users\\larsd\\OneDrive - TU Eindhoven\\Universiteit\\Jaar 3\\Kwartiel 3\\8P361 - Project Imaging\\Good data\\"
model_names = ["EfficientNet", "Inception", "ResNet", "MobileNetV2", "Xception"]
N = 57458

def ensemble(path, model_names, N, option=0):
    # option 0 = standard majority voting
    # option 1 = majority voting based on extreme probabilities
    # option 2 = standard averaging
    # option 3 = dynamic averaging
    
    data = {"id": [], "label": [ [] for _ in range(N) ]}
    for i in range(len(model_names)):
        model = model_names[i]
        filepath = path + model + "_submission.csv"
        csv_file = open(filepath)
        csv_reader = csv.reader(csv_file, delimiter=',')
        ids = []
        labels = []
        for row in csv_reader:
            ids.append(row[0])
            labels.append(row[1])
        ids = ids[1:]
        labels = labels[1:]
        for j in range(N):
            data["id"].append(ids[j])
            data["label"][j].append(float(labels[j]))
    data["id"] = list(dict.fromkeys(data["id"]))

    classes = []
    for i in range(N):
        sum_ = 0
        for j in range(5):
            sum_ += round(data["label"][i][j])
        classes.append(round(sum_/5))    
    
    preds = {"id": data["id"], "label": []}
    
    if option == 0:
        for i in range(N):
            avg = []
            for j in range(5):
                if round(data["label"][i][j]) == classes[i]:
                    avg.append(data["label"][i][j])
            avg = np.mean(avg)
            preds["label"].append(avg)  
    
    if option == 1:
        for i in range(N):
            if classes[i] == 0:
                preds["label"].append(min(data["label"][i]))
            else:
                preds["label"].append(max(data["label"][i])) 
    
    if option == 2:
        for i in range(N):
            avg = []
            for j in range(5):
                avg.append(data["label"][i][j])
            avg = np.mean(avg)
            preds["label"].append(avg)
    
    if option == 3:
        for i in range(N):
            w_preds = []
            certainties = []
            for j in range(5):
                if data["label"][i][j] >= 0.5:
                    certainties.append(data["label"][i][j])
                else:
                    certainties.append(1 - data["label"][i][j])
            for j in range(5):
                w = certainties[j]/sum(certainties)
                w_preds.append(w*data["label"][i][j])
            preds["label"].append(sum(w_preds))
            
    df = pd.DataFrame(preds)
    df = df.set_index("id")
    
    if option == 0:
        df.to_csv(r"C:\Users\larsd\OneDrive - TU Eindhoven\Universiteit\Jaar 3\Kwartiel 3\8P361 - Project Imaging\majority_voting_submission.csv")
    elif option == 1:
        df.to_csv(r"C:\Users\larsd\OneDrive - TU Eindhoven\Universiteit\Jaar 3\Kwartiel 3\8P361 - Project Imaging\majority_voting_extremes_submission.csv")
    elif option == 2:
        df.to_csv(r"C:\Users\larsd\OneDrive - TU Eindhoven\Universiteit\Jaar 3\Kwartiel 3\8P361 - Project Imaging\averaging_submission.csv")
    elif option == 3:
        df.to_csv(r"C:\Users\larsd\OneDrive - TU Eindhoven\Universiteit\Jaar 3\Kwartiel 3\8P361 - Project Imaging\averaging_dynamic_submission.csv")
        
ensemble(path, model_names, N, option=0)
ensemble(path, model_names, N, option=1)
ensemble(path, model_names, N, option=2)
ensemble(path, model_names, N, option=3)